In [ ]:
# This cell is only for setting example parameter defaults - gets replaced by sidecar.
day_obs_min = "2025-04-17"
day_obs_min = "Yesterday"
day_obs_max = "Today"
#day_obs_max = "2024-12-10"
#instrument = "latiss"  
#instrument = "lsstcomcam"
instrument = "lsstcam"

# ConsDB Visit Metadata from {{ params.day_obs_min }} to {{ params.day_obs_max }}

In [ ]:
import os
import copy
from math import floor
import numpy as np
import healpy as hp
import matplotlib.pylab as plt
from cycler import cycler
import colorcet as cc

import pandas as pd
from pandas import option_context
from IPython.display import display, Markdown, HTML

import datetime
from astropy.time import Time, TimeDelta
import astropy.units as u
import astropy
astropy.utils.iers.conf.iers_degraded_accuracy = 'ignore'


from rubin_scheduler.site_models import Almanac
from rubin_scheduler.utils import Site

In [ ]:
!pip install --user --upgrade git+https://github.com/lsst-sims/rubin_nights.git --no-deps > /dev/null 2>&1

import os
from rubin_nights.consdb_query import ConsDbFastAPI
from rubin_nights.connections import get_access_token
import rubin_nights.utils as rn_utils

In [ ]:
# test that day_obs_min is a proper day_obs
if day_obs_min.lower() == "today":
    day_obs_min = rn_utils.today_day_obs()
elif day_obs_min.lower() == "yesterday":
    day_obs_min = rn_utils.yesterday_day_obs()
else:
    # test that day_obs is a proper day_obs    
    try:
        test_day_obs = Time(f"{day_obs_min}T12:00:00", format='isot', scale='utc')
    except ValueError:
        msg = "day_obs_min should be a date formatted as YYYY-MM-DD"
        raise ValueError(msg)
    
if day_obs_max.lower() == "today":
    day_obs_max = rn_utils.today_day_obs()
elif day_obs_max.lower() == "yesterday":
    day_obs_max = rn_utils.yesterday_day_obs()
else:
    # test that day_obs is a proper day_obs    
    try:
        test_day_obs = Time(f"{day_obs_max}T12:00:00", format='isot', scale='utc')
    except ValueError:
        msg = "day_obs_max should be a date formatted as YYYY-MM-DD"
        raise ValueError(msg)

In [ ]:
minutes_to_days = 1./60/24
seconds_to_days = 1./60/60/24

day_min = Time(f"{day_obs_min}T12:00:00", format='isot', scale='utc')
day_max = Time(f"{day_obs_max}T12:00:00", format='isot', scale='utc')
one_day = TimeDelta(1, format='jd')
days = day_min + one_day * np.arange(0, (day_max - day_min).jd + 1)
day_obss = [d.iso[0:10] for d in days]

## Accessing visits from ConsDB (USDF)

In [ ]:
on_rsp = True
# Are you on an RSP?
if on_rsp:
    api_base = os.getenv("EXTERNAL_INSTANCE_URL", "")
    token = get_access_token()
# Or are you outside of an RSP? - just use USDF and your own token
# See https://rsp.lsst.io/guides/auth/creating-user-tokens.html
else:
    api_base = "https://usdf-rsp.slac.stanford.edu"
    token = get_access_token("/Users/lynnej/.lsst/usdf_rsp")
consdb = ConsDbFastAPI(api_base=api_base, auth=('user', token))

In [ ]:
visits = consdb.get_visits(instrument, day_min, day_max + one_day)

if len(visits) > 0:
    display(Markdown(f"Retrieved {len(visits)} visits from consdb from {day_min.iso} to {(day_max + one_day).iso}"))
if len(visits) == 0:
    display(Markdown(f"No visits for {instrument} between {day_obs_min} to {day_obs_max} retrieved from consdb"))

In [ ]:
## Almanac ## 
display(Markdown(f"## Daily information for {day_obs_min} to {day_obs_max}"))
site = Site('LSST')
almanac = Almanac()
alm = {}
for day_obs in day_obss:
    alm[day_obs] = {}
    night_events = almanac.get_sunset_info(evening_date=day_obs, longitude=site.longitude_rad)
    civil_sunset = Time(night_events['sunset'], format='mjd', scale='utc') 
    sunset = Time(night_events['sun_n12_setting'], format='mjd', scale='utc') 
    sunrise = Time(night_events['sun_n12_rising'], format='mjd', scale='utc')
    night_length = (sunrise.mjd - sunset.mjd) * 24
    alm[day_obs]['civil sunset'] = civil_sunset.iso
    alm[day_obs]['sunset'] = sunset.iso
    alm[day_obs]['sunrise'] = sunrise.iso
    alm[day_obs]['moon rise'] = Time(night_events['moonrise'], format='mjd', scale='utc').iso
    alm[day_obs]['moon set'] = Time(night_events['moonset'], format='mjd', scale='utc').iso
    moon_phase = almanac.get_sun_moon_positions(sunset.mjd)['moon_phase']
    alm[day_obs]['moon phase'] = moon_phase.round(2)
    dayobsint = int(day_obs.replace('-', ''))
    vv = visits.query('day_obs == @dayobsint')
    alm[day_obs]['programs'] = vv.science_program.unique()
    alm[day_obs]['nvisits dayobs'] = len(vv)
    visit_times = visits['exp_midpt']
    vv = visits.query('day_obs == @dayobsint and exp_midpt_mjd >= @civil_sunset.mjd and ~img_type.str.contains("BIAS") and ~img_type.str.contains("DARK")')
    alm[day_obs]['nvisits night'] = len(vv)
    alm[day_obs]['sunset to night visits'] = ((vv.exp_midpt_mjd.min() - sunset.mjd) * 24).round(3)
    alm[day_obs]['night visits to sunrise'] = ((sunrise.mjd - vv.exp_midpt_mjd.max()) * 24).round(3)
    alm[day_obs]['first to last visit @ night'] = round((vv.exp_midpt_mjd.max() - vv.exp_midpt_mjd.min()) * 24, 3)
    alm[day_obs]['night length'] = night_length.round(3)
alm = pd.DataFrame(alm)
print("Night visits are after civil sunset and not BIAS or DARK")
print("Time durations are in hours")
print("Sunset without qualifier is 12 degree sunset (or sunrise)")
with option_context('display.max_colwidth', None):
    display(HTML(alm.T.to_html()))

In [ ]:
c = None
if len(visits) > 0:
    visits['visit_id'] = visits.index.copy()
    groupcols = ['science_program', 'img_type', 'target_name', 'observation_reason', 'day_obs', 'visit_id'] 
    c = visits[groupcols].groupby(['science_program', 'img_type'], dropna=False).agg({'science_program' : ['first'],
                                                                        'target_name' : ['unique'], 
                                                                        'observation_reason' : ['unique'],
                                                                        'day_obs' : ['nunique'],
                                                                        'visit_id' : ['first', 'last', 'count']})

In [ ]:
# This might work .. to help translate test block numbers above into more meaningful programs
testcase_base_url = "https://rubinobs.atlassian.net/projects/BLOCK?selectedItem=com.atlassian.plugins.atlassian-connect-plugin:com.kanoah.test-manager__main-project-page#!/v2/testCase/"
jiraticket_base_url = "https://rubinobs.atlassian.net/browse/"

if len(visits) > 0:
    jira_urls = {}
    for science_program in c.sort_values(by=('visit_id', 'first'))[('science_program', 'first')].values: # visits.science_program.unique():
        if science_program is not None and science_program.startswith("BLOCK-T"):
            jira_urls[science_program] = testcase_base_url + science_program
        elif science_program is not None and science_program.startswith("BLOCK-"):
            jira_urls[science_program] = jiraticket_base_url + science_program
        else:
            jira_urls[science_program] = ''
            #display(Markdown(f"[{science_program}]({jira_url}) - {test_name} ({len(visits.query('science_program == @science_program'))} visits)"))

In [ ]:
jira_url_col = np.array(["X" *  (max([len(v) for v in jira_urls.values()])+100)] * len(c))
for i, (ri, row) in enumerate(c.iterrows()):
    sp = row[('science_program', 'first')]
    if sp in jira_urls.keys():
        jira_url_col[i] = f'<a href="{jira_urls[sp]}" target="_blank" rel="noreferrer noopener">{sp}</a>'
    else:
        jira_url_col[i] = ''
c['JIRA'] = jira_url_col

In [ ]:
if len(visits) > 0:
    display(Markdown(f"ConsDB Visits"))
    with option_context('display.max_colwidth', None):
        cols = [('JIRA', ''), ('target_name', 'unique'), ('observation_reason', 'unique'), ('day_obs', 'nunique'),
                ('visit_id', 'first'), ('visit_id', 'last'), ('visit_id', 'count')]
        display(HTML(c[cols].sort_values(by=('visit_id', 'first')).to_html(escape=False)))

## Visits vs Time properties

This is a visits vs time example from schedview, using better Rubin color mapping for the various filters.  Due to memory and size, this runs on the last three nights only. (download notebook and run yourself for more). 

In [ ]:
import bokeh
import bokeh.io

bokeh.io.output_notebook()

sys.path.insert(0, "/sdf/data/rubin/user/lynnej/repos/schedview")
import schedview.plot

if len(visits):
    dayobs_cutoff = int((day_max - TimeDelta(3, format='jd')).iso[0:10].replace('-', ''))
    vv = visits.query('day_obs > @dayobs_cutoff').copy()
    if len(vv):
        def make_time(x):
            return Time(x['exp_midpt'], format='isot', scale='tai').to_datetime()
        vv['start_date'] = vv.apply(make_time, axis=1)
        vv['filter'] = vv['band']
        if 'visit_id' in vv.columns:
            vv.drop('visit_id', axis=1, inplace=True)
        if "zero_point_median" in vv:
            vv['zero_point_median'] = vv['zero_point_median'].values.astype(float)
        col = 'psf_sigma_median'
        if 'psf_sigma_median' not in vv.columns:
            col = 'zenith_distance'
        plot = bokeh.plotting.figure(y_axis_label=col, x_axis_label="Time (UTC)", height=600, width=1200)
        fig = schedview.plot.plot_visit_param_vs_time(vv, col, show_column_selector=True, hovertool=False, plot=plot)
        bokeh.io.show(fig)
else:
    print("No visits")

In [ ]:
# cols = ['physical_filter', 'exp_midpt', 'band', 's_ra', 's_dec', 'sky_rotation', 'exp_time', 'airmass', 'dimm_seeing', 'psf_sigma_median', 'seeing_zenith_500nm_median', 'sky_bg_median', 'zero_point_median']
# sub = visits.query('target_name.str.contains("ECDFS") and shut_time>0')
# sub.query('shut_time < 30')[['science_program', 'observation_reason']]

In [ ]:
# import hvplot.pandas
# import panel as pn

# explorer = visits.hvplot.explorer(x='start_date', y=['psf_sigma_median'], by=['band'], groupby=['science_program'], 
#                                  kind='scatter', alpha=0.4, legend='bottom_right',
#                                  )
# explorer